# Event Finder

The goal of this notebook is to develop a generalized event finder. Based on user selected variable and event definitions, this notebook will search all simulations to find these events.  

Inputs:

* Domain, a custom shapefile
* One variable
* Low and/or High thresholds
* Duration 
* Global Warming Level

**Runtime**: With the default settings, this notebook takes approximately 25 minutes to run from start to finish. Modifications to selections may increase the runtime.

**Troubleshooting**: Kernel crashing when running get_data()? Try loading a smaller spatial area by setting the `latitude` and `longitude` variables or using a `cached_area`.

## Make Selections

Before running this notebook, make the following selections for variables, domain, and GWLs.

1. Select a timescale - this will impact which downscaling methods are available and which variables are available.

In [1]:
timescale = "daily"  # options are hourly, daily or monthly

2. Select a variable - the code below displays which variables are available based on your timescale options

In [2]:
# import function
from climakitae.explore.agnostic import show_available_vars

In [3]:
# display variables available for this analysis
# only showing dynamical because statistical has a smaller subset that is in the larger dynamical subset
# only difference is max/min RH is available in statistical and not in dynamical
show_available_vars("Dynamical", timescale)

['Air Temperature at 2m',
 'Precipitation (total)',
 'Relative humidity',
 'Dew point temperature',
 'Mean wind speed at 10m',
 'Maximum wind speed at 10m',
 'Surface Pressure',
 'Specific humidity at 2m',
 'Surface skin temperature',
 'Maximum air temperature at 2m',
 'Minimum air temperature at 2m',
 'Instantaneous downwelling longwave flux at bottom',
 'Instantaneous downwelling shortwave flux at bottom',
 'Shortwave flux at the surface',
 'Longwave flux at the surface',
 'Sensible heat flux at the surface',
 'Latent heat flux at the surface',
 'Ground heat flux',
 'Snowfall',
 'Liquid water path',
 'Evaporation',
 'Ice water path',
 'Maximum precipitation',
 'Effective Temperature']

In [7]:
# make selection from list above
event_variable = "Maximum wind speed at 10m"
event_units = "mph"

3. Define Event - Select a method for establishing the event

In [5]:
# define the function that will be used to calculate the event threshold
event_threshold_function = "1 in x"  # options are "percentile" or "1 in x"
event_threshold_direction = (
    ">"  # look for events above or below the result from the threshold function
)
event_threshold_function_params = (
    10  # if 1 in X include X, if quantile, include quantile (0.25, 0.5, etc.)
)
event_duration = 1  # how many consecutive events need to be observed to be a hit

4. Select a baseline and future GWL

In [6]:
# current options 0.8, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0, 4.0
baseline_gwl = "0.8"
future_gwl = "2.0"

5. Select Spatial Domain - Replace the ellipses with the path to your region shapefile and a name for your region.

For example, someone studying the Pajaro watershed might have a shapefile path that looks like this:
```
shapefile_filename = "PajaroRiverWatershed.zip"
spatial_domain = "Pajaro"  # name will appear in title
```

In [8]:
# edit path below and filename for shapefile to use
shapefile_filename = "~/the-shoppe/ventura/pcluster/VenturaRiverWatershed/VenturaRiverWatershed_WBDHUC10.shp" # replace ellipsis with filepath
spatial_domain_name = "ventura" # name will appear in title

6. Select Resolution - select the the model resolution (3km, 9km, 45 km)

In [9]:
resolution = "3 km"

7. Aggregation Method - select a method to aggregate spatially

In [10]:
aggregation = "max"  # options are mean, median, sum, min, max

## Import libraries & define functions

In [11]:
import climakitae as ck
from climakitae.core.data_interface import (
    get_data_options,
    get_subsetting_options,
    get_data,
)
from climakitae.util.utils import add_dummy_time_to_wl
from climakitae.explore import threshold_tools
from climakitae.explore.threshold_tools import (
    get_block_maxima,
    get_return_value,
    get_return_period,
)
from climakitae.core.data_load import load

import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [12]:
def clip_to_shapefile(da, shapefile):
    """
    Clip a DataArray to the boundaries of a shapefile.

    Parameters:
    - da: xarray DataArray to be clipped
    - shapefile: path to the shapefile

    Returns:
    - Clipped DataArray
    """
    # Read the shapefile using geopandas
    gdf = gpd.read_file(shapefile)

    # Ensure the DataArray has a CRS set
    # da.rio.write_crs("epsg:2228", inplace=True)

    # Clip the DataArray using the shapefile geometry
    clipped_da = da.rio.clip(gdf.geometry.apply(mapping), gdf.crs, drop=True)

    return clipped_da

## Step 1: Define Event 

Use the baseline GWL to define the event threshold

### Pull Baseline Data

In [13]:
latitude = (34.29642 - 0.04, 34.66964 + 0.04)
longitude = (-119.51431 - 0.04, -119.08539 + 0.04)

#### Dynamical Downscaling (WRF)

In [14]:
# WRF downscaling
wrf_data = get_data(
    variable=event_variable,
    downscaling_method="Dynamical",
    resolution=resolution,
    timescale=timescale,
    units=event_units,
    # Modify your approach
    approach="Warming Level",
    warming_level=float(baseline_gwl),
    latitude=latitude,
    longitude=longitude
)

In [15]:
# rename lat lon
wrf_data = wrf_data.rename({"x": "longitude", "y": "latitude"})

In [16]:
# add a dummy time variable
wrf_data = add_dummy_time_to_wl(wrf_data)

#### Statistical Downscaling (LOCA2)

In [17]:
# grab correct variable name for LOCA2
loca_variable = [event_variable]
# temperature
if loca_variable[0] == "Air Temperature at 2m":
    loca_variable = ["Maximum air temperature at 2m", "Minimum air temperature at 2m"]
# relative humidity
if loca_variable[0] == "Relative humidity":
    loca_variable = ["Minimum relative humidity", "Maximum relative humidity"]

In [18]:
# statistical - loca2 downscaling xvariable
loca_data = [
    get_data(
        variable=GETVAR,
        downscaling_method="Statistical",
        resolution=resolution,
        units=event_units,
        timescale=timescale,
        # Modify your approach
        approach="Warming Level",
        warming_level=float(baseline_gwl),
        latitude=latitude,
        longitude=longitude
    )
    for GETVAR in loca_variable
]

ERROR: No data found for your input values. Please modify your data request. 
Returning None


In [19]:
# if we need to average
if loca_data[0] is not None:
    # temperature
    if event_variable == "Air Temperature at 2m":
        loca_data = (loca_data[0] + loca_data[1]) / 2
        loca_data.name = "Air Temperature at 2m"

    # relative humidity
    if event_variable == "Relative humidity":
        loca_data = (loca_data[0] + loca_data[1]) / 2
        loca_data.name = "Relative humidity"

In [20]:
# unlist if needed
if type(loca_data) == list:
    loca_data = loca_data[0]

# now merge
# first check if there is valid loca data - requires that both x and y have valid data
if loca_data is None:
    # set to wrf
    loca_data = wrf_data
    # replace with all NaNs
    loca_data = loca_data * np.nan
else:
    loca_data = loca_data.rename({"lon": "longitude", "lat": "latitude"})
# add a dummy time variable
loca_data = add_dummy_time_to_wl(loca_data)

ValueError: DataArray does not contain necessary warming level information.

### Downsize Data

In [ ]:
# clip to shapefile
wrf_data = clip_to_shapefile(wrf_data, shapefile_filename)
loca_data = clip_to_shapefile(loca_data, shapefile_filename)

In [ ]:
# average across domain (we want single timeseries)
wrf_data = eval(f"wrf_data.{aggregation}(['longitude','latitude'])")
loca_data = eval(f"loca_data.{aggregation}(['longitude','latitude'])")

### Import Data

In [ ]:
loca_data = load(loca_data, progress_bar=True).squeeze()
wrf_data = load(wrf_data, progress_bar=True).squeeze()

### Define Event

In [ ]:
# determine what method to define event
if event_threshold_function == "percentile":
    # calculate the percentile
    loca_threshold = loca_data.quantile(
        dim="time", q=event_threshold_function_params
    ).squeeze()
    wrf_threshold = wrf_data.quantile(
        dim="time", q=event_threshold_function_params
    ).squeeze()
# if 1 in x
if event_threshold_function == "1 in x":
    if event_threshold_direction == ">":
        extreme_type = "max"
    else:
        extreme_type = "min"
    # initialize
    loca_list = []
    loca_sim_list = []
    for sim in loca_data.simulation:
        try:
            loca_ams = get_block_maxima(
                loca_data.sel({"simulation": sim}),
                check_ess=False,
                extremes_type=extreme_type,
            )
            loca_rv = get_return_value(
                loca_ams,
                return_period=event_threshold_function_params,
                multiple_points=False,
            )
            loca_rv = loca_rv.expand_dims(dim="simulation")
            loca_list.append(loca_rv)
            loca_sim_list.append(sim.data.item())
        except Exception as e:
            # Usually exception is due to missing data at chosen GWL such that
            # get_block_maxima cannot run. Skip this simulation.
            print(e)
            print("Skipping ", sim.data.item())
    loca_threshold = xr.concat(
        loca_list, loca_data.simulation.sel({"simulation": loca_sim_list})
    )
    loca_threshold = loca_threshold["return_value"].squeeze()
    # intalize
    wrf_list = []
    for sim in wrf_data.simulation:
        wrf_ams = get_block_maxima(
            wrf_data.sel({"simulation": sim}),
            check_ess=False,
            extremes_type=extreme_type,
        )
        wrf_rv = get_return_value(
            wrf_ams,
            return_period=event_threshold_function_params,
            multiple_points=False,
        )
        wrf_rv = wrf_rv.expand_dims(dim="simulation")
        wrf_list.append(wrf_rv)
    wrf_threshold = xr.concat(wrf_list, wrf_data.simulation)
    wrf_threshold = wrf_threshold["return_value"].squeeze()

In [ ]:
# remove baseline data to free up memory
del loca_data
del wrf_data

## Step 2: Estimate Event 'Hits'

Use the future GWL to estimate event hits

### Pull Future GWL Data

#### Dynamical Downscaling (WRF)

In [ ]:
# WRF downscaling
wrf_data = get_data(
    variable=event_variable,
    downscaling_method="Dynamical",
    resolution=resolution,
    timescale=timescale,
    units=event_units,
    # Modify your approach
    approach="Warming Level",
    warming_level=float(future_gwl),
)

In [ ]:
# rename lat lon
wrf_data = wrf_data.rename({"x": "longitude", "y": "latitude"})

In [ ]:
# add a dummy time variable
wrf_data = add_dummy_time_to_wl(wrf_data)

#### Statistical Downscaling (LOCA2)

In [ ]:
# grab correct variable name for LOCA2
loca_variable = [event_variable]
# temperature
if loca_variable[0] == "Air Temperature at 2m":
    loca_variable = ["Maximum air temperature at 2m", "Minimum air temperature at 2m"]
# relative humidity
if loca_variable[0] == "Relative humidity":
    loca_variable = ["Minimum relative humidity", "Maximum relative humidity"]

In [ ]:
# statistical - loca2 downscaling xvariable
loca_data = [
    get_data(
        variable=GETVAR,
        downscaling_method="Statistical",
        resolution=resolution,
        units=event_units,
        timescale=timescale,
        # Modify your approach
        approach="Warming Level",
        warming_level=float(future_gwl),
    )
    for GETVAR in loca_variable
]

In [ ]:
# if we need to average
if loca_data[0] is not None:
    # temperature
    if event_variable == "Air Temperature at 2m":
        loca_data = (loca_data[0] + loca_data[1]) / 2
        loca_data.name = "Air Temperature at 2m"

    # relative humidity
    if event_variable == "Relative humidity":
        loca_data = (loca_data[0] + loca_data[1]) / 2
        loca_data.name = "Relative humidity"

In [ ]:
# unlist if needed
if type(loca_data) == list:
    loca_data = loca_data[0]

# now merge
# first check if there is valid LOCA2 data - requires that both x and y have valid data
if loca_data is None:
    # set to wrf
    loca_data = wrf_data
    # replace with all NaNs
    loca_data = loca_data * np.nan
else:
    loca_data = loca_data.rename({"lon": "longitude", "lat": "latitude"})
# add a dummy time variable
loca_data = add_dummy_time_to_wl(loca_data)

### Downsize Data

In [ ]:
# clip to shapefile
wrf_data = clip_to_shapefile(wrf_data, shapefile_filename)
loca_data = clip_to_shapefile(loca_data, shapefile_filename)

In [ ]:
# average across domain (we want single timeseries)
wrf_data = eval(f"wrf_data.{aggregation}(['longitude','latitude'])")
loca_data = eval(f"loca_data.{aggregation}(['longitude','latitude'])")

### Import Data

In [ ]:
loca_data = load(loca_data, progress_bar=True).squeeze()
wrf_data = load(wrf_data, progress_bar=True).squeeze()

In [ ]:
# If simulation is missing in baseline dataset,
# drop from future. Seems to be a LOCA2 issue only.
loca_sim_list_new = list(loca_data.simulation.values)
loca_sim_list = [x for x in loca_sim_list if x in loca_sim_list_new]
loca_data = loca_data.sel({"simulation": loca_sim_list})

### Count Hits

In [ ]:
### create a 'hit' for in events
# initialize an array to fill
wrfEventHit = np.zeros(wrf_data.values.shape)
locaEventHit = np.zeros(loca_data.values.shape)

# loop through each time stamp
for itime in range(0, len(wrf_data["time"])):
    # look at this time stamp + duration
    timeIndex = list(range(itime, (itime + event_duration)))
    # remove any values that are greater than our time
    timeIndex = [x for x in timeIndex if x < len(wrf_data["time"])]

    ### Start with WRF
    # pull out the data to test
    testWRFData = wrf_data[timeIndex, :]

    # for each simulation, check if the values are greater or less than the threshold
    if event_threshold_direction == "<":
        wrfCounts = (testWRFData < wrf_threshold).sum(dim="time")
    if event_threshold_direction == ">":
        wrfCounts = (testWRFData > wrf_threshold).sum(dim="time")

    # if you have greater than or equal to duration count as a 'hit'
    wrfHitIndex = wrfCounts >= event_duration

    # now save hits
    wrfEventHit[itime, wrfHitIndex] = 1

    ### Move to Loca
    # pull out the data to test
    testLOCAData = loca_data[timeIndex, :]

    # for each simulation, check if the values are greater or less than the threshold
    if event_threshold_direction == "<":
        locaCounts = (testLOCAData < loca_threshold).sum(dim="time")
    if event_threshold_direction == ">":
        locaCounts = (testLOCAData > loca_threshold).sum(dim="time")

    # if you have greater than or equal to duration count as a 'hit'
    locaHitIndex = locaCounts >= event_duration

    # now save hits
    locaEventHit[itime, locaHitIndex] = 1

In [ ]:
# now add hits to xarray
wrf_data = wrf_data.assign_coords(event_hit=(("time", "simulation"), wrfEventHit))
loca_data = loca_data.assign_coords(event_hit=(("time", "simulation"), locaEventHit))

## Prep for Plotting

In [ ]:
# create a 'model' variable
lsims = loca_data.simulation.values.tolist()
loca_models = [s.split("_")[1] for s in lsims]
loca_data = loca_data.assign_coords(models=("simulation", loca_models))

In [ ]:
wsims = wrf_data.simulation.values.tolist()
wrf_models = [s.split("_")[1] for s in wsims]
wrf_data = wrf_data.assign_coords(models=("simulation", wrf_models))

In [ ]:
# create a realization variable
loca_realization = [s.split("_")[2] + "_" + s.split("_")[3] for s in lsims]
loca_data = loca_data.assign_coords(relization=("simulation", loca_realization))
wrf_realization = [s.split("_")[2] + "_" + s.split("_")[3] for s in wsims]
wrf_data = wrf_data.assign_coords(relization=("simulation", wrf_realization))

In [ ]:
# create a data frame to make plotting easier
locaDF = loca_data.to_dataframe().reset_index()
wrfDF = wrf_data.to_dataframe().reset_index()

In [ ]:
# Create a downscaling name
locaDF["downscaling"] = "loca2"
wrfDF["downscaling"] = "wrf"

In [ ]:
# combine into one
finalDF = pd.concat([locaDF, wrfDF], keys=locaDF.keys())

## Step 3: Plot Results (Yay!)

In [ ]:
# plot all the data
def annotate(data, **kws):
    ax = plt.gca()
    ax.scatter(
        data.time.where(data.event_hit == 1),
        data[event_variable].where(data.event_hit == 1),
        color="r",
    )


g1 = sns.relplot(
    data=finalDF,
    x="time",
    y=event_variable,
    col="simulation",
    color="grey",
    col_wrap=4,
    kind="line",
    height=5,
    aspect=1,
    facet_kws=dict(sharex=False),
)
g1.map_dataframe(annotate)
# g1.map(sns.scatterplot,data=finalDF[finalDF["event_hit"] == 1],
#              x="time", y=event_variable, color='r')

In [ ]:
# Save the figure to a file
if event_threshold_function == "1 in x":
    event_name = f"1_in_{event_threshold_function_params}"
else:
    event_name = event_threshold_function_params + "_percentile"
g1.savefig(
    f"event_finder_{event_variable}_under_{future_gwl}gwl_with_{event_name}_{event_duration}{timescale}duration.jpeg".replace(
        " ", "_"
    )
)

In [ ]:
# save event data
finalDF.to_parquet(
    f"event_{event_variable}_under_{future_gwl}gwl_with_{event_name}_{event_duration}{timescale}duration".replace(
        " ", "_"
    )
)